In [ ]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D
import scipy as sp
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from scipy.interpolate import make_interp_spline
%matplotlib inline
import plotly.express as px
import utils as NFLUtils
from PIL import Image

In [ ]:
eval_frame_df = pd.read_csv('./data/eval_frame_df.csv')
eval_tracking_df = pd.read_csv('./data/eval_tracking_df.csv')
tackles_df = pd.read_csv('./data/tackles.csv')

eval_tracking_df['displayId'] = eval_tracking_df['displayName'] + eval_tracking_df['nflId'].map(str)

In [ ]:
def plot_cyc_time_series(gameId:int, playId:int, player:int, eval_frame_df:pd.DataFrame, save_path="./test_animation.gif", frames_dir='./frames'): 
    frames_dir = f'./frames_{gameId}_{playId}'
    games_df = pd.read_csv('data/games.csv')   
    game = games_df.loc[(games_df['gameId'] == gameId)] 
    homeTeamAbbr = game['homeTeamAbbr'].iloc[0] 
    visitorTeamAbbr = game['visitorTeamAbbr'].iloc[0]
    file_path = f"{gameId}_{homeTeamAbbr}_{visitorTeamAbbr}" 
    tpc = pd.read_csv(f'games/{file_path}/{playId}/tpc_per_frame_weighted_blockers.csv', index_col=0)    
    # cpr = pd.read_csv(f'games/{file_path}/{playId}/cpr_weighted_blockers.csv', index_col=0)    
    eval_frame_df_filtered = eval_frame_df.loc[(eval_frame_df['gameId'] == gameId) & (eval_frame_df['playId'] == playId)].copy()
    # print(eval_frame_df_filtered) 
    # print(tpc.head())  

    eval_frame_df_filtered['CPR'] = tpc[str(player)].values
    # eval_frame_df_filtered['cpr'] = cpr[str(player)].values
    eval_frame_df_filtered['CYC'] = tpc[str(player)].values * eval_frame_df_filtered['expectedYardsByCarrier'].diff(-1).fillna(0)
    eval_frame_df_filtered['EYD'] = eval_frame_df_filtered['expectedYardsByCarrier'].diff(-1).fillna(0).copy()

    # Define figure size
    fig_size = (12, 8)  # Adjust as needed

    # Determine axis limits
    x_lim = (eval_frame_df_filtered['frameId'].min(), eval_frame_df_filtered['frameId'].max())
    y_lim = (eval_frame_df_filtered[['CPR', 'CYC', 'EYD']].min().min(),
            eval_frame_df_filtered[['CPR', 'CYC', 'EYD']].max().max())

    # Iterate over each frame in the DataFrame
    for frame in eval_frame_df_filtered['frameId'].unique():
        fig, ax = plt.subplots(figsize=fig_size)
        current_data = eval_frame_df_filtered[eval_frame_df_filtered['frameId'] <= frame]
        
        for metric in ['CPR', 'CYC', 'EYD']:
            x = current_data['frameId']
            y = current_data[metric]

            # Exponential moving average for smoothing
            span = 5  # Adjust as needed
            y_smooth = y.ewm(span=span, adjust=False).mean()

            # Plot settings for each metric
            if metric == 'CPR':
                color, linestyle = 'blue', 'dashed'
            elif metric == 'CYC':
                color, linestyle = 'green', 'solid'
            elif metric == 'EYD':
                color, linestyle = 'red', 'dashed'

            # Plotting
            ax.plot(x, y_smooth, label=f'{metric} (smooth)', color=color, linestyle=linestyle)
            ax.plot(x.iloc[-1], y_smooth.iloc[-1], 'o', color=color)  # Dot at the end

        ax.set_xlim(x_lim)
        ax.set_ylim(y_lim)
        ax.legend()
        plt.title("Constrictive Presence Ratio (CPR), Expected Yardage Differential (EYD), and Carrier Yards Constricted (CYC) over Frames")
        plt.xlabel('Frame ID')
        plt.ylabel('Value')

        # Save the frame
        plt.savefig(os.path.join(frames_dir, f'frame_{frame}.png'))
        plt.close()

    # Longer duration for slower animation
    frame_duration = 200  # Duration in milliseconds

    # Create an animation from the saved frames
    images = [Image.open(os.path.join(frames_dir, f)) for f in sorted(os.listdir(frames_dir)) if f.endswith('.png')]
    images[0].save(save_path, save_all=True, append_images=images[1:], duration=frame_duration, loop=0)



    

In [ ]:
plot_cyc_time_series(gameId=2022092502, playId=3510, player=42406, eval_frame_df=eval_frame_df, save_path='./test_graph.gif') # CHANGE THIS!
